In [2]:
import pandas as pd
from data import CassavaDataset
from pathlib import Path
from augmentations import get_augmentations,get_tta
from torch.utils.data import DataLoader
from lightning import CassavaModel
import torch
from models import Resnext

In [5]:
ls ../data/

cassava-leaf-disease-classification.zip  test_tfrecords/  train_tfrecords/
label_num_to_disease_map.json            train.csv        valid_df.pkl
sample_submission.csv                    train_df.pkl
test_images/                             train_images/


In [6]:
test_df = pd.read_csv('../data/sample_submission.csv')
path = Path('../data/')
batch_size,num_workers = 64,8
ssl_models = [
    "resnet18_ssl",
    "resnet50_ssl",
    "resnext50_32x4d_ssl",
    "resnext101_32x4d_ssl",
    "resnext101_32x8d_ssl",
    "resnext101_32x16d_ssl",
]

In [7]:
test_df

,image_id,label
0,2216849948.jpg,4


In [8]:
tta_tfms = get_tta(image_size=512)
test_ds = CassavaDataset(path=path/'test_images',df=test_df,transform=tta_tfms)


In [9]:
test_dl = DataLoader( dataset=test_ds,
            batch_size=batch_size,
            num_workers=num_workers,
            shuffle=False,
            pin_memory=True,)

In [10]:
for xb,_ in test_dl:
    break

In [11]:
xb.shape

torch.Size([1, 3, 512, 512])

In [12]:
device = torch.device('cuda')
model = Resnext(model_name=ssl_models[2],num_classes=5,kaggle=True)
model = model.to(device)


Using cache found in /root/.cache/torch/hub/facebookresearch_semi-supervised-ImageNet1K-models_master


In [14]:
for xb,_ in test_dl:
    xb = xb.to(device)
    preds = model(xb)


In [15]:
preds.shape

torch.Size([1, 5])

In [17]:
ls -al 'Cassava Leaf Disease'/

total 4
drwxr-xr-x 4 root root   38 Dec 14 06:33 ./
drwxr-xr-x 7 root root 4096 Dec 14 10:07 ../
drwxr-xr-x 3 root root   25 Dec 14 06:33 1qip6pkk/
drwxr-xr-x 3 root root   25 Dec 14 05:50 cz4hh30s/
